In [26]:
import torch
import dqc
import time
import pyscf
from pyscf import gto, dft, cc
import numpy as np 

In [3]:
def DQC_HF(system, basis):
    atomzs, atomposs = dqc.parse_moldesc(system)
    atomposs
    mol = dqc.Mol((atomzs, atomposs), basis=basis)
    qc = dqc.HF(mol).run()
    ene = qc.energy()  # calculate the energy
    
    return ene

In [4]:
def PySCF_HF(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    hf = scf.HF(mol)
    ene = hf.kernel(verbose=0)
    return ene

In [5]:
def DQC_KS_PBE(system, basis):
    atomzs, atomposs = dqc.parse_moldesc(system)
    atomposs
    mol = dqc.Mol((atomzs, atomposs), basis=basis)
    qc = dqc.KS(mol, xc="gga_c_pbe").run()
    ene = qc.energy() # calculate the energy
    return ene

In [6]:
def PySCF_KS_PBE(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    ks = dft.RKS(mol)
    ks.xc = "pbe"
    ene = ks.kernel()
    return ene

In [7]:
def PySCF_CCSDT(system, basis):
    mol = gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    hf = scf.HF(mol).run()
    cc_ = cc.CCSD(hf)
    ene = cc_.kernel()
    et = cc_cc.ccsd_t()
    return cc_.e_tot + et

In [8]:
def print_timer(t):
    print(f' took {round(time.time() - t, 3)} s.')

In [11]:
def accuracy_comparison(system, basis):
    t = time.time()
    print("HF, DQC: ", DQC_HF(system, basis))
    print_timer(t)
    t = time.time()
    print("HF, PySCF: ", PySCF_HF(system, basis))
    print_timer(t)
    t = time.time()
    print("PBE, DQC: ", DQC_KS_PBE(system, basis))
    print_timer(t)
    t = time.time()
    print("PBE, PySCF: ", PySCF_KS_PBE(system, basis))
    print_timer(t)
    t = time.time()
    print("CCSD(T): ", PySCF_CCSDT(system, basis))
    print_timer(t)

Hydrogen at equlibrium point:

In [12]:
accuracy_comparison("H 0 0 0; H 1.4 0 0", "6-31G")

HF, DQC:  tensor(-1.1267, dtype=torch.float64)
 took 0.011 s.
HF, PySCF:  -1.1267427044517944
 took 0.047 s.
PBE, DQC:  tensor(-0.5625, dtype=torch.float64)
 took 0.086 s.
PBE, PySCF:  -1.1619157577671007
 took 0.083 s.
CCSD(T):  -1.1516791660767662
 took 0.074 s.


In [9]:
%timeit DQC_HF("H 0 0 0; H 1.4 0 0", "6-31G")

8.56 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit PySCF_HF("H 0 0 0; H 1.4 0 0", "6-31G")

51.2 ms ± 750 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit DQC_KS_PBE("H 0 0 0; H 1.4 0 0", "6-31G")

93.2 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit PySCF_KS_PBE("H 0 0 0; H 1.4 0 0", "6-31G")

94.4 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Nitrogen at equlibrium point:

In [13]:
accuracy_comparison("N 0 0 0; N 2.07 0 0", "6-31G")

HF, DQC:  tensor(-108.8679, dtype=torch.float64)
 took 0.024 s.
HF, PySCF:  -108.86787508005207
 took 0.053 s.
PBE, DQC:  tensor(-96.5082, dtype=torch.float64)
 took 0.216 s.
PBE, PySCF:  -109.35334923417093
 took 0.15 s.
CCSD(T):  -109.102646799969
 took 0.22 s.


In [12]:
%timeit DQC_HF("N 0 0 0; N 2.07 0 0", "6-31G")

22.6 ms ± 236 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit PySCF_HF("N 0 0 0; N 2.07 0 0", "6-31G")

57.2 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%timeit DQC_KS_PBE("N 0 0 0; N 2.07 0 0", "6-31G")

220 ms ± 5.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit PySCF_KS_PBE("N 0 0 0; N 2.07 0 0", "6-31G")

154 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Ammonia at equlibrium point:

In [14]:
accuracy_comparison("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

HF, DQC:  tensor(-56.1610, dtype=torch.float64)
 took 0.033 s.
HF, PySCF:  -56.1610237839644
 took 0.063 s.
PBE, DQC:  tensor(-49.0140, dtype=torch.float64)
 took 0.4 s.
PBE, PySCF:  -56.45136347268183
 took 0.217 s.
CCSD(T):  -56.292149164904174
 took 0.183 s.


In [15]:
%timeit DQC_HF("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

25.6 ms ± 805 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit PySCF_HF("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

67.3 ms ± 974 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%timeit DQC_KS_PBE("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

401 ms ± 9.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit PySCF_KS_PBE("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

217 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Benzene at equlibrium point:

In [53]:
benzene_coords = """C        0.598362921      0.000000000     -4.742986733;
                    C       -0.557705772     -0.354690359     -4.044822733;
                    C        1.754431614      0.354690359     -4.044822733;
                    H       -1.457130878     -0.630640582     -4.587995733;
                    H        2.653856720      0.630640582     -4.587995733;
                    C       -0.557705772     -0.354690359     -2.648492733;
                    C        1.754431614      0.354690359     -2.648492733;
                    H       -1.457130878     -0.630640582     -2.105319733;
                    H        2.653856720      0.630640582     -2.105319733;
                    C        0.598362921      0.000000000     -1.950328733;
                    H        0.598362921      0.000000000     -0.863981733;
                    H        0.598362921      0.000000000     -5.829333733"""

In [17]:
accuracy_comparison(benzene_coords, "6-31G")

HF, DQC:  tensor(-216.7411, dtype=torch.float64)
 took 0.956 s.
HF, PySCF:  -216.74175971295512
 took 0.354 s.
PBE, DQC:  tensor(-179.7154, dtype=torch.float64)
 took 5.676 s.
PBE, PySCF:  -218.09557954674182
 took 2.252 s.
CCSD(T):  -217.07797043633224
 took 6.179 s.


In [19]:
%timeit DQC_HF(benzene_coords, "6-31G")

971 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit PySCF_HF(benzene_coords, "6-31G")

317 ms ± 7.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit DQC_KS_PBE(benzene_coords, "6-31G")

5.85 s ± 147 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit PySCF_KS_PBE(benzene_coords, "6-31G")

2.23 s ± 32.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Naphtalene at equlibrium point:

In [18]:
naphtalene_coords = """H       -1.057571674      0.000000000     -4.361571149;
                    C       -1.057571674      0.487597890     -3.390034963;
                    C       -1.057571674      1.860916946     -3.299493774;
                    C       -1.057571674      2.493399263     -2.031832408;
                    C       -1.057571674      1.739878824     -0.880133713;
                    C       -1.057571674      0.320033769     -0.937967730;
                    C       -1.057571674     -0.320033769     -2.220831877;
                    C       -1.057571674     -1.739878824     -2.278665893;
                    C       -1.057571674     -2.493399263     -1.126967199;
                    C       -1.057571674     -1.860916946      0.140694168;
                    C       -1.057571674     -0.487597890      0.231235357;
                    H       -1.057571674      2.466184811     -4.201283252;
                    H       -1.057571674      3.577893185     -1.973126484;
                    H       -1.057571674      2.222891720      0.093690092;
                    H       -1.057571674      0.000000000      1.202771542;
                    H       -1.057571674     -2.466184811      1.042483645;
                    H       -1.057571674     -3.577893185     -1.185673122;
                    H       -1.057571674     -2.222891720     -3.252489699"""

In [19]:
accuracy_comparison(naphtalene_coords, "6-31G")

HF, DQC:  tensor(-359.6521, dtype=torch.float64)
 took 6.055 s.
HF, PySCF:  -359.6605290994828
 took 2.358 s.
PBE, DQC:  tensor(-298.3350, dtype=torch.float64)
 took 31.809 s.
PBE, PySCF:  -361.94173753829205
 took 11.143 s.
CCSD(T):  -360.24270680988127
 took 97.77 s.


In [83]:
%timeit DQC_HF(naphtalene_coords, "6-31G")

12.7 s ± 1.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
%timeit PySCF_HF(naphtalene_coords, "6-31G")

4.31 s ± 1.18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%timeit DQC_KS_PBE(naphtalene_coords, "6-31G")

32.4 s ± 945 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit PySCF_KS_PBE(naphtalene_coords, "6-31G")

17.2 s ± 4.29 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Anthracene at equlibrium point:

In [23]:
anthracene_coords = """C        1.044178831     -0.528233237     -4.684523771;
C        1.044178831     -0.552713285     -3.254782652;
C        1.044178831     -1.749044624     -2.526815306;
C        1.044178831      0.704131514     -2.541488804;
C        1.044178831      0.692578077     -1.141127549;
C        1.044178831     -0.503753261     -0.413160203;
C        1.044178831     -1.760598060     -1.126454051;
C        1.044178831      0.658108897     -5.368920348;
C        1.044178831      1.919058175     -3.295618865;
H        1.044178831      1.638563069     -0.604255156;
C        1.044178831     -0.528233309      1.016580916;
C        1.044178831     -2.975524721     -0.372323990;
C        1.044178831     -2.954793300      0.997119891;
C        1.044178831     -1.714574573      1.700977987;
H        1.044178831      0.418178836      1.551045394;
H        1.044178831     -3.919661467     -0.910797789;
H        1.044178831     -3.885538136      1.556731594;
H        1.044178831     -1.717772077      2.786998206;
C        1.044178831      1.898326754     -4.665062745;
H        1.044178831     -1.474645383     -5.218988249;
H        1.044178831     -2.695029615     -3.063687699;
H        1.044178831      0.661305531     -6.454941061;
H        1.044178831      2.863194921     -2.757145066;
H        1.044178831      2.829071589     -5.224674449"""

In [24]:
t = time.time()
print("HF, DQC: ", DQC_HF(anthracene_coords, "6-31G"))
print_timer(t)

HF, DQC:  tensor(-502.5513, dtype=torch.float64)
 took 21.373 s.


In [25]:
t = time.time()
print("HF, PySCF: ", PySCF_HF(anthracene_coords, "6-31G"))
print_timer(t)

HF, PySCF:  -502.5894981125605
 took 7.291 s.


In [26]:
t = time.time()
print("PBE, PySCF: ", PySCF_KS_PBE(anthracene_coords, "6-31G"))
print_timer(t)

PBE, PySCF:  -505.80427292380455
 took 38.487 s.


In [27]:
t = time.time()
print("PBE, DQC: ", DQC_KS_PBE(anthracene_coords, "6-31G"))
print_timer(t)

/home/rolan/miniconda3/envs/noa/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=1.182e-01, |f|=3.977e-01 at iter 28
  warnings.warn(ConvergenceWarning(msg))


PBE, DQC:  tensor(-416.9599, dtype=torch.float64)
 took 123.793 s.


In [28]:
t = time.time()
print("CCSD(T): ", PySCF_CCSDT(anthracene_coords, "6-31G"))
print_timer(t)

CCSD(T):  -503.42435350874797
 took 1089.783 s.


In [3]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

In [2]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) 
                          + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
a = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a, p)

In [30]:
def dqc_compute(system, basis="6-31G"):
    mol = dqc.Mol(moldesc=system, basis=basis)
    qc = dqc.KS(mol, xc=myxc).run()
    ene = qc.energy()
    return ene.detach()

In [35]:
def my_eval_xc(a,p):    
    def eval_xc(xc_code, rho, spin=0, relativity=0, deriv=1, verbose=None):
        exc = a * rho**p
        vrho = a * p * rho**(p-1)
        vgamma = None
        vlapl = None
        vtau = None
        vxc = (vrho, vgamma, vlapl, vtau)
        fxc = None  # 2nd order functional derivative
        kxc = None  # 3rd order functional derivative
        return exc, vxc, fxc, kxc
    return eval_xc

In [48]:
def scf_compute(system, basis="6-31G"):
    scf_mol = pyscf.gto.M(atom = system, basis=basis, unit="Bohr")
    mf = pyscf.dft.RKS(scf_mol)
    mf = mf.define_xc_(my_eval_xc(1,2), 'LDA')
    mf.verbose = 0
    return mf.kernel()

In [55]:
scf_compute(benzene_coords)

10.257178847011062

In [56]:
dqc_compute(benzene_coords)

/home/rolan/miniconda3/envs/noa/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=8.325e-02, |f|=2.075e-01 at iter 31
  warnings.warn(ConvergenceWarning(msg))


tensor(-82.6614, dtype=torch.float64)

In [59]:
%timeit scf_compute(benzene_coords)

3.21 s ± 40.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%timeit dqc_compute(benzene_coords)

10.2 s ± 287 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
